# 主题向量



本章将学习这些语义或主题向量，通过TF-IDF向量的加权得分来计算所谓的主题得分，而将这些得分构成了主题向量的各个维度。

将使用归一化词频直接的关联来将词归并到同意主题，每个归并结果定义了新主题向量的一个维度。
----
本章的学习主要包括以下部分
* 根据nlp流水线，实现文本的读取、清洗、分词、以及tf-idf向量化的过程
* 基于tf-idf文本向量化矩阵，实现主题向量分析，
> tf-idf文本向量从本质而言只是从统计词频上体现了文本的特征，但缺少对文本含义的反映。
我们需要一种方法来从词的统计数据中提取一些额外信息，即意义信息。需要更好地估算文档中的词到底意味着什么，也需要知道这些词的组合在一篇具体的文档中意味着什么，我们需要对tf-idf文本向量矩阵进一步处理，得到新的向量更加紧凑，更有意义。
p
**主题向量**
主题向量可以对应词的意义向量：“词-主题向量”（word-topic vector），以及文档意义向量“文档-主题向量”（document-topic vector）。两者都额可以称为主题向量，不过表现对象一个是词一个是文档。
简单地理解就是将文本矩阵进行降维，行（文本数量）不变，列（原为统一后分词数，例如1000个）根据算法降低为新的列（主题数），使得文本能对应不同主题。

主题分析涉及多种不同的方法，包括：

1. 通过隐形语义分析（latent senmantic analysis LSA）可以不仅仅把词的意义表示为向量，还可以用向量来表示通篇文档的意义。
>LSA是一种分析tf-idf文本向量矩阵的算法，他将词分组到主题中，LSA也可以对词袋向量进行处理，但是tf-idf向量给出的结果更好。
>LSA 还可以对这些主题进行优化，以保持主题维度的多样性。当使用这些新主题而不是原始词时，我们依然可以捕获文档的大部分含义（语义）。该模型中用于捕获文档含义所需要主题数量远远少于tf-idf向量词汇表中词的数量。
>LSA通常被认为是一种降维技术，减少捕获文档含义所需要的维数。
>其数学背景为PCA以及SVD，实际上，由于tf-idf矩阵不太可能是方阵，故使用的是SVD。
2. 线性判别分析（linear discriminant analysis，LDA）(不深度讲解)
3. 潜在狄利克雷分布(Latent Dirichlet Allocation,LDiA)
>LDiA还可以用来生成捕捉词或文档语义的向量。LDiA和LSA在数学上并不一样，它使用非线性统计算法将词分组。因此，它通常会比线性方法（如LSA）的训练时间长很多。这常常使LDiA在许多实际应用中并不实用。尽管如此，它所创建的主题的统计数据有时更接近于人类对词和主题的直觉，所以LDiA主题通常更易于向上级解释。此外，LDiA对于一些单文档问题有用，如文档摘要。这时，语料库就是单篇文档，文档的句子就是该“语料库”中的一篇篇“文档”。这就是gensim和其他软件包使用LDiA来识别文档中最核心句子的做法。然后这些句子可以串在一起形成一篇由机器生成的摘要[11]。对于大多数分类或回归问题，通常最好使用LSA。但结合可视化时其主要使用的是LDiA算法

In [ ]:
import numpy as np
import random
topic = {}
random.seed(1)
tfidf = dict(
    list(zip('cat dog apple lion NYC love'.split(), np.random.rand(6))))

print("得到虚拟的每个词的tf-idf值")
tfidf
topic['petness'] = (0.3*tfidf['cat']+0.3*tfidf['dog']+0 *
                    tfidf['apple']+0*tfidf['lion']-0.2*tfidf['NYC']+0.2*tfidf['love'])
topic['animalness'] = (0.1*tfidf['cat']+0.1*tfidf['dog']-0.1 *
                       tfidf['apple']+0.5*tfidf['lion']+0.1*tfidf['NYC']-0.1*tfidf['love'])
topic['cityness'] = (0*tfidf['cat']-0.1*tfidf['dog']+0.2*tfidf['apple'] -
                     0.1*tfidf['lion']-0.5*tfidf['NYC']+0.1*tfidf['love'])
print(topic)
# 构建相应矩阵
topic_m = np.zeros(shape=(3, 6))
print(topic_m)
word_tf = np.zeros(shape=(6, 1))
print(word_tf)


### LDA 

LDA 分类器是一种有监督算法，因此需要对文本进行标注，但是其需要训练的样本数相对较少。

LDA是一维模型，所以其不需要SVD，可以只计算二类问题（如垃圾和非垃圾）问题中的每一类的所有TF-IDF向量的质心（平均值）。推导就变成了这两个质心之间的直线，TF-IDF向量与这条直线越近（TF-IDF向量与这两条直线的点积）就表示它与其中一个类接近。

In [ ]:
from cgitb import handler
import re
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression as LR
from sklearn.model_selection import cross_val_score
import numpy as np
print("处理训练数据：...\n")
train_txt = pd.read_table('sms/train.txt', sep='\t', header=None)
train_txt.columns = ['label', 'text']
label_map = {'ham': 0, 'spam': 1}  # 1为垃圾短信
train_txt['label'] = train_txt['label'].map(label_map)

# train_txt = pd.get_dummies(train_txt, columns=['label'])# 将标签onehot编码


def pre_clean_text(origin_text):
    # 去掉标点符号和非法字符
    text = re.sub("[^a-zA-Z]", " ", origin_text)
    # 将字符全部转化为小写，并通过空格符进行分词处理
    words = text.lower().split()

    # 将剩下的词还原成str类型
    cleaned_text = " ".join(words)

    return cleaned_text


if __name__ == '__main__':

    # 清理数据
    train_txt['text'] = train_txt['text'].apply(lambda x: pre_clean_text(x))

    # 删去空值.测试时若无效词删去后为空则直接为垃圾信息(实际测试中没有)
    # print(train_txt.shape)
    train_txt = train_txt.loc[train_txt['text'] != '', :]
    # 查看数据

    # print(train_txt.shape)
    # 实现tf-id数据向量化

    tfidf = TfidfVectorizer(
        analyzer="word",
        tokenizer=None,
        preprocessor=None,
        stop_words=None,
        max_features=200)
    word_vict = tfidf.fit_transform(train_txt['text']).toarray()
    print(word_vict.shape)
    print(word_vict[20, :])

    mask = np.array(train_txt['label'].astype(bool))
    print("得到{}矩阵".format(mask.shape))
    spam_centroid = word_vict[mask].mean(axis=0).round(2)  # axis=0 计算列平均值
    print("垃圾短信平均向量：", spam_centroid.shape)
    ham_centroid = word_vict[mask].mean(axis=0).round(2)

    print("短信平均向量：", ham_centroid.shape)
    sh = spam_centroid-ham_centroid
    print(sh.shape)
    spamsocre = word_vict@sh

    spamscore2 = word_vict@ham_centroid

    from sklearn.preprocessing import MinMaxScaler
    spam1 = MinMaxScaler().fit_transform(
        spamsocre.reshape(-1, 1))  # reshape(-1,1)转换成1列：

    spam2 = MinMaxScaler().fit_transform(spamscore2.reshape(-1, 1))

    train_txt['lda_score'] = spam1
    train_txt['lda_pred'] = (train_txt['lda_score'] > 0.2).astype(int)
    train_txt


# 隐性语义分析(LSA,Latent Semantic Analysis)
> 参考
> * https://zhuanlan.zhihu.com/p/46376672
> * https://zhuanlan.zhihu.com/p/144367432

LSA 是主题分析中最重要的技术,是主题分析讲解的重点
LSA的底层是SVD技术，利用SVD将TF_IDF矩阵分解3个矩阵，而后根据其方差贡献率（信息载荷）进行降维，当在NLL中使用SVD时，将其称为隐性语义分析（LSA），

LSA揭示了被隐含并等待被发现的词的语义或意义。

LSA是一种属性技术，用于寻找对任意一组NLP向量进行最佳线性变换（旋转和拉伸）的方法，这些NLP向量包括TF-IDF向量或词袋向量。对许多应用而言，最好的变换方法是将

坐标轴（维度）对齐到新向量中，使得其在词频上具有最大方差。然后可以在新向量空间中去掉哪些对不同文档向量贡献不大的维度。

<font color='red'>LSA 中单词-文本-svd关系</font>
<img src="https://pic2.zhimg.com/v2-b3eb29a45d1fc11f57b858fd5af7571d_r.jpg"/>
> LSA 步骤
1. 构建TF-IDF或其他文档-词矩阵向量,行为文档(doc)，列为词(term)
<img src="https://pic4.zhimg.com/80/v2-288292d4fd98b748c4b5e37786c06bd3_720w.jpg"/>
2. 对矩阵向量进行SVD分解
3. 根据主题数目，或方差贡献率累计比，选择降维数目
4. 对原有矩阵进行降维


---
潜在语义分析基于最古老和最常用的降维技术——奇异值分解（SVD）。SVD甚至早在“机器学习”这个术语出现之前就已经广泛使用。SVD将一个矩阵分解成3个方阵，其中一个是对角矩阵。SVD的一个应用是求逆矩阵。一个矩阵可以分解成3个更简单的方阵，然后对这些方阵求转置后再把它们相乘，就得到了原始矩阵的逆矩阵。

SVD是一种可以将任何矩阵分解成3个因子矩阵的算法，而这3个因子矩阵可以相乘来重建原始矩阵。这类似于为一个大整数找到恰好3个整数因子，但是这里的因子不是标量整数，而是具有特殊性质的二维实矩阵。通过SVD计算出的3个因子矩阵具有一些有用的数学性质，这些性质可以用于降维和LSA。在线性代数课上，我们可能已经用过SVD来求逆矩阵。这里，我们将使用它为LSA计算出主题***（相关词的组合）***。无论是在基于词袋的词项-文档矩阵还是基于TF-IDF的词项-文档矩阵上运行SVD，SVD都会找到属于同类的词组合。SVD通过计算词项-文档矩阵的列（词项）之间的相关度来寻找那些同时出现的词。SVD能同时发现文档间词项使用的相关性和文档之间的相关性。利用这两条信息，SVD还可以计算出语料库中方差最大的词项的线性组合。这些词项频率的线性组合将成为我们的主题。我们将只保留那些在语料库中包含信息最多、方差最大的主题。SVD同时也提供了词项-文档向量的一个线性变换（旋转），它可以将每篇文档的向量转换为更短的主题向量。

In [ ]:
!pip install nltk
!pip install jieba

以下为直接通过np中SVD方法进行降维，在实际应用中不会用到这种方法，但通过代码可以了解背后的数学含义。

In [ ]:
from sklearn.datasets import fetch_20newsgroups
import pandas as pd
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer as tf
import numpy as np
from sklearn.decomposition import TruncatedSVD
import jieba
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

def loadData():
    '''实现载入sklearn中的“20 Newsgroup”数据
    '''
    dataset = fetch_20newsgroups(
        shuffle=True, random_state=1, remove=('headers', 'footers', 'quotes'))
    return pd.DataFrame(dataset.data)


def clearData(df: pd.DataFrame):
    """开始之前，我们先尝试着清理文本数据。主要思想就是清除其中的标点、数字和特殊字符。之后，我们需要删除较短的单词，因为通常它们不会包含什么有用的信息。最后，我们将文本变为不区分大小写。

    Args:
        df (pd.DataFrame): _原始文本_
    """
# removing everything except alphabets`

    df['clean_doc'] = df['document'].str.replace("[^a-zA-Z#]", " ")
# removing short words
    df['clean_doc'] = df['clean_doc'].apply(
        lambda x: ' '.join([w for w in x.split() if len(w) > 3]))
# make all text lowercase
    df['clean_doc'] = df['clean_doc'].apply(lambda x: x.lower())
    return df


def cutChineseWord(data, stopwordfile):

    stopwords = [line.strip() for line in open(
        stopwordfile, 'r', encoding='utf-8').readlines()]
    # 首先实现切词
    for i in range(len(data)):
        doc = data.iloc[i-1, 0]
        seq_list = jieba.cut(doc, use_paddle=True)
        token = []

        for seq in seq_list:
            if seq not in stopwords:
                token.append(seq)

        data.at[i, 'words'] = list(token)  # 每次在words列第i个位置插入对象list
    return data


def stopWords(df: pd.DataFrame):
    """之后我们要删除没有特别意义的停止词，例如“it”、“they”、“am”、“been”、“about”、“because”、“while”等等。为了实现这一目的，我们要对文本进行标记化，也就是将一串文本分割成独立的标记或单词。删除停止词之后，再把这些标记组合在一起。

    Args:
        df (pd.DataFrame): _description_
    """
    stop_words = stopwords.words('english')
    # tokenization分词
    tokenized_doc = df['clean_doc'].apply(lambda x: x.split())
    # remove stop-words
    tokenized_doc = tokenized_doc.apply(
        lambda x: [item for item in x if item not in stop_words])
# de-tokenization
    detokenized_doc = []
    for i in range(len(df)):
        t = ' '.join(tokenized_doc[i])
        detokenized_doc.append(t)

    df['clean_doc'] = detokenized_doc
    return df


def vec_words(document: pd.DataFrame, max_feature: int = 200) -> np.array:
    """基于sklearn实现文档向量化，其中元素为tf-idf，注意TfidfVectorizer参数

    Args:
        document (pd.DataFrame): _description_

    Returns:
        _type_: _description_
    """
    tfidf = tf(analyzer="word", tokenizer=None, max_features=max_feature)
    tfidf.fit(document)
    lexcion = tfidf.vocabulary_  # 返回向量化对应的词典（字典形式）
    lexcion_len = tfidf.vocabulary_.__len__()  # 返回向量化对应的词典长度
    vec = tfidf.fit_transform(document).toarray()
    df = pd.DataFrame(vec, columns=list(lexcion.keys()))
    return vec, lexcion, lexcion_len, df, tfidf


def svd(vec: np.array):
    """基于np中线性代数函数实现svd，注意np.linalg.svd（）返回的特征值矩阵是一个向量而非矩阵，这是由于方法自动缩减了特征值
    矩阵中0值，k的大小为A(m*n)中最小的值，为此本方法将特征值矩阵还原为m*n大小的矩阵其中k个特征值构成其对角值。

    Args:
        vec (np.array): _description_

    Returns:
        _type_: _description_
    """

    u, sigma, vt = np.linalg.svd(vec)

    Sigma = np.zeros(np.shape(vec))
    len_ar = len(sigma)
    Sigma[:len_ar, :len_ar] = np.diag(sigma)
    return u, Sigma, vt, sigma


def RightdemsionRe(A: np.array, u: np.array, Sigma: np.array, k: int) -> np.array:
    """_实现基于SVD的文本矩阵降维,由于是进行主题降维度、所以采用左奇异矩阵将维，实现词的降维_
        SVD 中左（行）奇异矩阵降维 为 B=Ut（k）*A  其中Ut为U左奇异矩阵转置，k为转置后保留的行数

    Args:
        A(np.array):_原文档-词矩阵_
        u (np.array): _左奇异矩阵_
        Sigma (np.array): _奇异值矩阵_
        vt (np.array): _右奇异矩阵_
        n (int): _降维数目（话题数）_

    Returns:
        np.array: _description_
    """
    B = u.T[:k, :]@A
    # B=u[:,:n]@Sigma[:n,:n]@vt[:n,:]

    return B


In [ ]:
if __name__=="__main__":
    print("转为Data数据观察")
    documents=loadData()
    print(documents.head(3))
    print(documents.describe())
    #clear data
    documents.columns=['document']
    documents= clearData(documents)
    print(documents.head(3))
    # tf-idf 向量化 
    vec,lexcion,lexcion_len,df,tfidf=vec_words(documents['clean_doc'])
    print(type(lexcion))
    print("得到向量化矩阵形状(行：文本，列：词）：{},词典个数：{}".format(vec.shape,lexcion_len))
    vect=vec.T# 对词向量转置
    print("得到向量化矩阵形状(行：词，列：文本）：{},词典个数：{}".format(vect.shape,lexcion_len))
    print("词典 \n",lexcion)
    #print("向量化后文本(行：文本，列：词\n ",vec[:8,:])
    print("向量化后的文本矩阵(行：文本，列：词）：",df)
    print("在进行svd之前，由于我们的最终目标是进行主题降维，使用np.linga.svd构建u，sigma,vt时候 \n u对应的行向量就应该是其词（最初主题），故应该先对文本向量进行转置，输入（行：词，列：文本）矩阵，\n 在基础上进行左奇异矩阵降维")

    print("由于输入的向量化文本为行：text，列：word，但np.linalg.svd对应的原始矩阵为，行：word，列：text故需要先转置")
    u,Sigma,vt,sigma=svd(vect)
    print('sigma:',sigma.shape)
    print("svg后的左奇异矩阵大小:{},奇异值矩阵大小:{},右奇异矩阵大小：{},奇异矩阵中对角线奇异值数量:{}".format(u.shape,Sigma.shape,vt.shape,sigma.shape))
    B=RightdemsionRe(vect,u,Sigma,20)
    print("降维后数据大小：",B.shape)
    print(B[:,15])
    
    ### 通过sklearn中TruncatedSVD函数实现svd，但注意，其输入向量矩阵为（行：文本，列：词）
    svd_model = TruncatedSVD(n_components=20, algorithm='randomized', n_iter=100, random_state=122)

    sk_vec=svd_model.fit(vec)
    print("使用sklearn中TruncatedSVD进行降维，注意矩阵形态")
    print(len(svd_model.components_))
    
    '''
    svd_model中的元素就是我们的主题，
    我们可以用svdmodel.components查看。最后，在20个主题中输入几个重要单词，看模型会做出什么反应。
    '''
    terms=tfidf.get_feature_names()
    print(terms)
    print("#############################")
    print(lexcion)
    
    # 查看主题中信息
    for i, comp in enumerate(svd_model.components_):
        terms_comp = zip(terms, comp)
    sorted_terms = sorted(terms_comp, key= lambda x:x[1], reverse=True)[:7]
    print("Topic "+str(i)+": ")
    for t in sorted_terms:
        print(t[0])
        print(" ")
        

In [ ]:
!pip install openpyxl


基于sklearn来实现LSA主题分析，并且针对中文文档实现降维操作。并实现NLP处理的API流水线方式
> 参考资料
* [TfidfVectorizer](https://zhuanlan.zhihu.com/p/166636681#:~:text=%E5%85%B6%E4%B8%AD%20vectorizer.get_feature_names%20%28%29%20%E5%8C%85%E5%90%AB%E4%BA%86%E6%95%B0%E6%8D%AE%E4%B8%AD%E5%87%BA%E7%8E%B0%E7%9A%84%E6%89%80%E6%9C%89%E5%8D%95%E8%AF%8D%E5%8E%BB%E9%87%8D%E5%90%8E%E7%9A%84%E9%9B%86%E5%90%88%EF%BC%8C%E7%9B%B8%E5%BD%93%E4%BA%8E%E4%B8%80%E4%B8%AA%E8%AF%8D%E5%85%B8%E3%80%82,%E5%BD%93%E7%84%B6%E4%BD%A0%E4%B9%9F%E5%8F%AF%E4%BB%A5%E7%BB%99%20CountVectorizer%20%E6%8F%90%E4%BE%9B%E4%B8%80%E4%B8%AA%E5%8D%95%E7%8B%AC%E7%9A%84%E8%AF%8D%E5%85%B8%EF%BC%8C%E5%90%A6%E5%88%99%20CountVectorizer%20%E4%BC%9A%E8%87%AA%E5%B7%B1%E4%BB%8E%E6%95%B0%E6%8D%AE%E4%B8%AD%E5%AD%A6%E4%B9%A0%E5%88%B0%E8%AF%8D%E5%85%B8%E3%80%82)

In [ ]:
from multiprocessing.resource_sharer import stop
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
import numpy as np 
import jieba 
def stopwordslist(filepath):
     stopwords = [line.strip() for line in open(filepath, 'r',encoding='utf-8').readlines()]
     return stopwords
 
def vec_words(document: pd.DataFrame,max_features=200,output=True) -> np.array:
    """基于sklearn实现文档向量化，其中元素为tf-idf，注意TfidfVectorizer参数

    Args:
        document (pd.DataFrame): _description_
        max_features(int): 词典大小
    Returns:
        _type_: _description_
    """
    
    tfidf = TfidfVectorizer(token_pattern=r"(?u)\b\w+\b", max_df=0.6,max_features=max_features).fit(document)  
    tfidf.fit(document)
    lexcion = tfidf.vocabulary_  # 返回向量化对应的词典（字典形式）
    lexcion_len = tfidf.vocabulary_.__len__()  # 返回向量化对应的词典长度
     
    vec_origal=tfidf.fit_transform(document)# 为了实现可视化，还需要返回tfidf化文本的原始形式
    vec=vec_origal.toarray()#返回数组化后的tfidf矩阵
    df = pd.DataFrame(vec, columns=list(lexcion.keys()))
    vect=vec.T# 对词向量转置
    terms=tfidf.get_feature_names()
    #其中 vectorizer.get_feature_names()包含了数据中出现的所有单词去重后的集合，相当于一个词典。
    # 当然你也可以给 CountVectorizer 提供一个单独的词典，否则 CountVectorizer 会自己从数据中学习到词典。
    if output==True:
        print("得到向量化矩阵形状(行：文本，列：词）：{},词典个数：{}".format(vec.shape,lexcion_len))
        
        print("得到向量化矩阵形状(行：词，列：文本）：{},词典个数：{}".format(vect.shape,lexcion_len))
        print("词典 \n",lexcion)
        print("数据中出现所有单词去重后集合 \n",terms)
        #print("向量化后文本(行：文本，列：词\n ",vec[:8,:])
        print("向量化后的文本矩阵(行：文本，列：词）：",df)
    return vec, lexcion, lexcion_len, df, tfidf,terms,vec_origal

def cutDoc2list(df:pd.DataFrame,stopwordfile:str,user_dic:str)->list:
    """自定义切词方法，返回新的切词列以及包含切词结果的list

    Args:
        df (pd.DataFrame): 输入包含文本的DataFrame
        stopwordfile (string): 停用词目录
        user_dic (string): 用户字典目录

    Returns:
        list: 返回切词后的list
    """
    
    stopwords=stopwordslist(stopwordfile)
    stopwords.append("x000D")
    stopwords.append("报警人")
    
    jieba.load_userdict(user_dic)#用户定义词典
    df['content']
    df['words']='' #新增一列words
    df['words']=df['words'].astype('object') #先转换成object类型
    for i in range(len( df['content'])):
        doc= df['content'][i]
  
        token=[]
        seq_list=jieba.cut(doc)
        for word  in seq_list:
            if word not in stopwords:
              token.append(word)
    #print(token)
            df.at[i,'words']=list(token)  # 每次在words列第i个位置插入对象list
    word_list=list(df['words'])
    doc_list=[" ".join(s) for s in word_list]
    return df,doc_list
   


if __name__ == '__main__':
    #实现数据读取以及切词处理
    df_0 = pd.read_excel("data/TOP25.xlsx")
    stopwordfile = 'data/baidu_stopwords.txt'
 
    user_dic='data/dic.txt'
    df_0['content']= df_0['workorder_content'].astype(str)# 要保证切词前数据全部为str
    df,doc_list=cutDoc2list(df_0,stopwordfile,user_dic)
    #####切词后矩阵生成tf-idf矩阵
    vec,lexcion,lexcion_len,df,tfidf,terms,vec_origal=vec_words(doc_list)


[中文lSA参考1](https://cloud.tencent.com/developer/article/1530432)

使用TruncatedSVD,把原先规模为(文本数，词汇数)的特征矩阵X化为规模为(文本数，主题数)的新特征矩阵X2：

(由于主题数一般比词汇数少，这一方法也可以用来降维，以进行分类或聚类操作)

In [ ]:
from sklearn.decomposition import TruncatedSVD

# SVD represent documents and terms in vectors 
topics_num=15#设置主题数目
svd_model = TruncatedSVD(n_components=topics_num, algorithm='randomized', n_iter=100, random_state=122)

svd_model.fit(vec)
svd_matrix=svd_model.fit_transform(vec)
len(svd_model.components_)
print("降维后矩阵",svd_matrix)
print("降维后矩阵大小",svd_matrix.shape)
print("svd_matrix[i,t]为第i篇文档在第t个主题上的分布，所以该值越高的文档i，可以认为在主题t上更有代表性，我们便以此筛选出最能代表该主题的文档。")
#通过svd后的矩阵，根据文档在t个主题对应的相关数值，得到与第t个主题最相关的n_pic_docs个文档
n_pick_docs= 2# 返回相关的文档数目
topic_docs_id = [svd_matrix[:,t].argsort()[:-(n_pick_docs+1):-1] for t in range(topics_num)]#根据排序得到最相关的文档
topic_docs_id
print("降维后矩阵大小",svd_matrix.shape)

获取每个主题代表性的文档

In [ ]:
for i in range(len(topic_docs_id)):
    doc=topic_docs_id[i]
    doc1=doc[0]
    doc2=doc[1]
    print("第",i,"个主题对应的文档编号为：")
    print(topic_docs_id[i])
    print(df_0.iloc[doc1,2])
    print(df_0.iloc[doc2,2])

获取每个主题关键字,以及对应的文档

In [ ]:
n_pick_keywords = 4#
topic_keywords_id = [svd_model.components_[t].argsort()[:-(n_pick_keywords+1):-1] for t in range(topics_num)]
topic_keywords_id
for t in range(topics_num):
    print("topic %d:" % t)
    print("    keywords: %s" % ", ".join(terms[topic_keywords_id[t][j]] for j in range(n_pick_keywords)))
    for i in range(n_pick_docs):
        print("    doc %d" % i)
        print("\t"+doc_list[topic_docs_id[t][i]])

###　

主题的可视化分析
[参考1](https://blog.csdn.net/m0_49263811/article/details/122216150)

```python

import pyLDAvis

data = pyLDAvis.sklearn.prepare(sklearn_lda, sklearn_dtm, vectorizer)
#data = pyLDAvis.gensim.prepare(gensim_lda, gensim_dtm, dictionary)
#data = pyLDAvis.graphlab.prepare(topic_model, docs)
``` 

参数解读：



sklearnlda/gensimlda: 计算好的话题模型

sklearndtm/gensimdtm: 文档词频矩阵

vectorizer/dictionary: 词语空间

topic_model: graphlab生成的话题模型

docs: 语料集



In [ ]:
!pip install pyLDAvis

In [ ]:
###########可视化

import pyLDAvis
import pyLDAvis.sklearn
from sklearn.decomposition import LatentDirichletAllocation

lda_tfidf = LatentDirichletAllocation(n_components=15, random_state=0)
lda_tfidf.fit(vec_origal)
pyLDAvis.enable_notebook()
pic = pyLDAvis.sklearn.prepare(lda_tfidf, vec_origal, tfidf)
pyLDAvis.display(pic)
pyLDAvis.save_html(pic, 'lda_pass'+str(topics_num )+'.html')
#去工作路径下找保存好的html文件
 
 

实现
其中 vectorizer.get_feature_names()包含了数据中出现的所有单词去重后的集合，相当于一个词典。当然你也可以给 CountVectorizer 提供一个单独的词典，否则 CountVectorizer 会自己从数据中学习到词典。

In [ ]:
doc_cut_list=[list(jieba.cut(sent)) for sent in doc_list]

doc=[" ".join(s) for s in doc_cut_list]

vec=TfidfVectorizer(token_pattern=r"(?u)\b\w+\b") #  token_pattern这个参数使用正则表达式来分词，其默认参数为r"(?u)\b\w\w+\b"，其中的两个\w决定了其匹配长度至少为2的单词，所以这边减到1个。对这个参数进行更多修改，可以满足其他要求，比如这里依然没有得到标点符号， 

model=vec.fit(doc)
lexcion=model.vocabulary_
print("通过slearn建立的词典：",lexcion)
print("词典长度",len(lexcion))
sparse_result = model.transform(doc)
print(sparse_result)
sklearn_matrix=sparse_result.todense().round(2)
print(sklearn_matrix)

In [ ]:
from collections.abc import Mapping
from nlpia.book.examples.ch04_catdog_lsa_3x6x16 import word_topic_vectors
print(word_topic_vectors.shape)
